In [1]:
import torch
import torch
torch.cuda.empty_cache()
import torch.nn as nn
import torchvision
import torchvision.transforms as transform
import torch.nn.functional as F

import matplotlib.pyplot as plt
import numpy as np
import math


device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)

batch_size = 2

img_transform = transform.Compose([transform.ToTensor()   ]) 

train_set = torchvision.datasets.ImageFolder('/home/jaoks/Desktop/proyecto6IA/dataset/train', transform=transform.ToTensor())
val_set = torchvision.datasets.ImageFolder('/home/jaoks/Desktop/proyecto6IA/dataset/val', transform=transform.ToTensor())

print(len(train_set))
print(len(val_set))


cuda:0
1370
340


In [2]:
hd_resol_train = torch.utils.data.Subset(train_set, range(685))
low_resol_train = torch.utils.data.Subset(train_set, range(685, 1370))

hd_resol_val = torch.utils.data.Subset(val_set, range(0, 119))
low_resol_val = torch.utils.data.Subset(val_set, range(170, 289))

hd_resol_test = torch.utils.data.Subset(val_set, range(119, 170))
low_resol_test = torch.utils.data.Subset(val_set, range(289, 340))

In [3]:
train_loader_low = torch.utils.data.DataLoader(dataset=low_resol_train, batch_size=batch_size, shuffle=True)
train_loader_hd = torch.utils.data.DataLoader(dataset=hd_resol_train, batch_size=batch_size, shuffle=True) 

val_loader_low =  torch.utils.data.DataLoader(dataset=low_resol_val, batch_size=batch_size, shuffle=True)
val_loader_hd =  torch.utils.data.DataLoader(dataset=hd_resol_val, batch_size=batch_size, shuffle=True)

test_loader_low =  torch.utils.data.DataLoader(dataset=low_resol_test, batch_size=17, shuffle=True)
test_loader_hd =  torch.utils.data.DataLoader(dataset=hd_resol_test, batch_size=17, shuffle=True)

In [4]:
conv1 = nn.Conv2d(in_channels=3, out_channels=64, kernel_size=3, stride=1, padding=0)
pool1 = nn.MaxPool2d(2, stride = 2, return_indices=True)
conv2 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, stride=1, padding=0)
pool2 = nn.MaxPool2d(2, stride = 2, return_indices=True)
conv3 = nn.Conv2d(in_channels=128, out_channels=256, kernel_size=3, stride=1, padding=0)
pool3 = nn.MaxPool2d(2, stride = 2, return_indices=True)
conv4 = nn.Conv2d(in_channels=256, out_channels=512, kernel_size=5, stride=1, padding=0)

tempimg, _ = train_set[0]
tempimg = tempimg.unsqueeze(0)
out1 = F.relu(conv1(tempimg))
print("SALIDA CONV1")
print(out1.shape)
out1p, ind1 = pool1(out1)
print("SALIDA POOL")
print(out1p.shape)
out2 = F.relu(conv2(out1p))
print("SALIDA CONV2")
print(out2.shape)
out2p, ind2=pool2(out2)
print("SALIDA POOL")
print(out2p.shape)
out3 = F.relu(conv3(out2p))
print("SALIDA CONV3")
print(out3.shape)
out3p, ind3=pool3(out3)
print("SALIDA POOL")
print(out3p.shape)
out4 = F.relu(conv4(out3p))
print("SALIDA CONV4")
print(out4.shape)


SALIDA CONV1
torch.Size([1, 64, 254, 254])
SALIDA POOL
torch.Size([1, 64, 127, 127])
SALIDA CONV2
torch.Size([1, 128, 125, 125])
SALIDA POOL
torch.Size([1, 128, 62, 62])
SALIDA CONV3
torch.Size([1, 256, 60, 60])
SALIDA POOL
torch.Size([1, 256, 30, 30])
SALIDA CONV4
torch.Size([1, 512, 26, 26])


In [5]:
convTran1 = nn.ConvTranspose2d(in_channels=512,out_channels=256, kernel_size=5, stride=1, padding=0)
poolT1 = nn.MaxUnpool2d(kernel_size=2, stride=2)
convTran2 = nn.ConvTranspose2d(in_channels=256*2,out_channels=128, kernel_size=3, stride=1, padding=0)
poolT2 = nn.MaxUnpool2d(kernel_size=2, stride=2)
convTran3 = nn.ConvTranspose2d(in_channels=128*2,out_channels=64, kernel_size=3, stride=1, padding=0)
poolT3 = nn.MaxUnpool2d(kernel_size=2, stride=2)
convTran4 = nn.ConvTranspose2d(in_channels=64*2,out_channels=3, kernel_size=3, stride=1, padding=0)

out_1 = F.relu(convTran1(out4))
print("OUTPUT CT1")
print(out_1.shape)
print("OUTPUT UNPOOL")
print(ind3.shape)
out_1p = poolT1(out_1, ind3)
print(out_1p.shape)
out_1p = torch.cat([out_1p, out3], 1)
print("OUTPUT CT2")
out_2 = F.relu(convTran2(out_1p))
print(out_2.shape)
print("OUTPUT UNPOOL")
print(ind2.shape)
out_2p = poolT2(out_2, ind2, output_size= out2.size())
print(out_2p.shape)
out_2p = torch.cat([out_2p, out2], 1)
print("OUTPUT CT3")
out_3 = F.relu(convTran3(out_2p))
print(out_3.shape)
print("OUTPUT UNPOOL")
print(ind1.shape)
out_3p = poolT3(out_3, ind1)
print(out_3p.shape)
out_3p = torch.cat([out_3p, out1], 1)
print("OUTPUT CT4")
out_4 = F.relu(convTran4(out_3p))
print(out_4.shape)


OUTPUT CT1
torch.Size([1, 256, 30, 30])
OUTPUT UNPOOL
torch.Size([1, 256, 30, 30])
torch.Size([1, 256, 60, 60])
OUTPUT CT2
torch.Size([1, 128, 62, 62])
OUTPUT UNPOOL
torch.Size([1, 128, 62, 62])
torch.Size([1, 128, 125, 125])
OUTPUT CT3
torch.Size([1, 64, 127, 127])
OUTPUT UNPOOL
torch.Size([1, 64, 127, 127])
torch.Size([1, 64, 254, 254])
OUTPUT CT4
torch.Size([1, 3, 256, 256])


In [6]:
class Encoder(nn.Module):
  def __init__(self):
    super(Encoder, self).__init__()
    self.conv1 = nn.Conv2d(in_channels=3, out_channels=64, kernel_size=3, stride=1, padding=0)
    self.pool1 = nn.MaxPool2d(kernel_size = 2, stride = 2, return_indices=True)
    self.bn1 = nn.BatchNorm2d(64)
    self.conv2 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, stride=1, padding=0)
    self.pool2 = nn.MaxPool2d(kernel_size = 2, stride = 2, return_indices=True)
    self.conv3 = nn.Conv2d(in_channels=128, out_channels=256, kernel_size=3, stride=1, padding=0)
    self.pool3 = nn.MaxPool2d(2, stride = 2, return_indices=True)
    self.conv4 = nn.Conv2d(in_channels=256, out_channels=512, kernel_size=5, stride=1, padding=0)
    self.sig4 = nn.Sigmoid()

  def forward(self, image):
    out1 = F.relu(self.bn1(self.conv1(image)))
    out1p, ind1 = self.pool1(out1)
    out2 = F.relu(self.conv2(out1p))
    out2p, ind2 = self.pool2(out2)
    out3 = F.relu(self.conv3(out2p))
    out3p, ind3 = self.pool3(out3)
    out4 = self.sig4(self.conv4(out3p))
    z = out4
    return z, out1, ind1, out2, ind2, out3, ind3

class Decoder(nn.Module):
  def __init__(self):
    super(Decoder, self).__init__()
    self.convTran1 = nn.ConvTranspose2d(in_channels=512,out_channels=256, kernel_size=5, stride=1, padding=0)
    self.poolT1 = nn.MaxUnpool2d(kernel_size=2, stride=2)
    self.convTran2 = nn.ConvTranspose2d(in_channels=256*2,out_channels=128, kernel_size=3, stride=1, padding=0)
    self.poolT2 = nn.MaxUnpool2d(kernel_size=2, stride=2)
    self.convTran3 = nn.ConvTranspose2d(in_channels=128*2,out_channels=64, kernel_size=3, stride=1, padding=0)
    self.poolT3 = nn.MaxUnpool2d(kernel_size=2, stride=2)
    self.convTran4 = nn.ConvTranspose2d(in_channels=64*2,out_channels=3, kernel_size=3, stride=1, padding=0)

    
  def forward(self, latent, out1, ind1, out2, ind2, out3, ind3):
    out = F.relu(self.convTran1(latent))
    out = self.poolT1(out, ind3)
    out = torch.cat([out, out3], 1)
    out = F.relu(self.convTran2(out))
    out = self.poolT2(out, ind2, output_size= out2.size())
    out = torch.cat([out, out2], 1)
    out = F.relu(self.convTran3(out))
    out = self.poolT3(out, ind1)
    out = torch.cat([out, out1], 1)
    out = F.relu(self.convTran4(out))
    
    return out

In [7]:
class Autoencoder(nn.Module):
   def __init__(self):
        super(Autoencoder, self).__init__()
        self.encoder = Encoder()
        self.decoder = Decoder()

   def forward(self, x):
        latent, out1, ind1, out2, ind2, out3, ind3 = self.encoder(x)
        x_recon = self.decoder(latent, out1, ind1, out2, ind2, out3, ind3)
        return  x_recon

In [8]:
def train(model, train_loader, train_loader_hd, val_loader, val_loader_hd, Epochs, loss_fn):
    train_loss_avg = []
    val_loss_avg = []
    for epoch in range(Epochs):
        train_loss_avg.append(0)
        num_batches = 0
        iterator = iter(train_loader_hd)
        for image_batch, _ in train_loader:
            image_batch = image_batch.to(device)
            batch_list = next(iterator)
            image_batch_hd = batch_list[0].to(device)
            image_batch_recon = model(image_batch)
            loss = loss_fn(image_batch_recon, image_batch_hd)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
          
            train_loss_avg[-1] += loss.item()
            num_batches += 1
        
        train_loss_avg[-1] /= num_batches
        print('Epoch [%d / %d] average reconstruction error: %f' % (epoch+1, Epochs, train_loss_avg[-1]))
        
        iterator = iter(val_loader_hd)
        val_loss_avg.append(0)
        num_batches=0
        for image_batch, _ in val_loader:
            image_batch = image_batch.to(device)
            batch_list = next(iterator)
            image_batch_hd = batch_list[0].to(device)
            image_batch_recon = model(image_batch)
            with torch.no_grad():
                loss = loss_fn(image_batch_recon, image_batch_hd)
                val_loss_avg[-1] += loss.item()
                num_batches += 1
        val_loss_avg[-1] /= num_batches
        print('Epoch [%d / %d] average reconstruction validation error: %f' % (epoch+1, Epochs, val_loss_avg[-1]))
                
    return train_loss_avg, val_loss_avg

In [9]:
capacity = 64
epochs = 100    
learning_rate = 0.001
autoencoder = Autoencoder()
autoencoder.to(device)
loss_fn = nn.MSELoss()
optimizer = torch.optim.Adam(params=autoencoder.parameters(), lr=learning_rate, weight_decay=1e-6)


loss_result, loss_val = train(autoencoder,train_loader_low, train_loader_hd, val_loader_low, val_loader_hd, epochs, loss_fn)

RuntimeError: CUDA out of memory. Tried to allocate 32.00 MiB (GPU 0; 3.81 GiB total capacity; 64.25 MiB already allocated; 48.19 MiB free; 68.00 MiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [ ]:
for image_batch, _ in train_loader_low:
  image_batch = image_batch.to(device)
  image_batch_recon = autoencoder(image_batch)
  #Show_imgs(image_batch[0].cpu(),"")
  #Show_imgs(image_batch_recon[0].cpu(),"")
  #image_batch[0].cpu() 
  im = transform.ToPILImage()(image_batch[0]).convert("RGB")  
  display(im)
  imt = transform.ToPILImage()(image_batch_recon[0]).convert("RGB")
  display(imt) 
  break